# Draft notebook:  HEASARC acces through python
This notebook goes through examples of how to access HEASARC data, whether catalogs, images, or spectra, using various VO protocols.  The various services are described on the __[HEASARC VO](https://heasarc.gsfc.nasa.gov/docs/archive/vo/)__ page.  

In [157]:
import matplotlib
%matplotlib inline  

## Example 1:  _astroquery.heasarc_

The _astroquery.heasarc_ package is a place-holder HEASARC package with currently limited functionality.  This example is from its __[help page](https://astroquery.readthedocs.io/en/latest/heasarc/heasarc.html)__:

In [159]:
from astroquery.heasarc import Heasarc
heasarc = Heasarc()
mission = 'rospublic'
object_name = '3c273'
table = heasarc.query_object(object_name, mission=mission)
table[:3].pprint()

   SEQ_ID   INSTRUMENT EXPOSURE ... PUBLIC_DATE  SEARCH_OFFSET_
                          S     ...     MJD                    
----------- ---------- -------- ... ----------- ---------------
RH701576N00 HRI           68154 ...       50186  0.192 (3C273)

RP600242A01 PSPCB         24822 ...       50437 34.236 (3C273)

RH700234N00 HRI           17230 ...       50312  0.192 (3C273)



astropy.table.table.Table

## Example 2:   __[HEASARC VO](https://heasarc.gsfc.nasa.gov/docs/archive/vo/instructions.html#_accessing)__  example with SkyView and SIA:

This example uses the generic python package _requests_ and the __[SkyView](https://skyview.gsfc.nasa.gov)__ example from the __[HEASARC VO](https://heasarc.gsfc.nasa.gov/docs/archive/vo/instructions.html#_accessing)__ page on how to submit a VO request to HEASARC.  This is equivalent to the request:

https://skyview.gsfc.nasa.gov/cgi-bin/vo/sia.pl?survey=sfd&POS=270.67,-22.97&SIZE=3 


### Sending the query

In [160]:
import requests
#  Coordinates don't come out Galactic.  Grid does nothing.  
#payload = {'survey': 'dss', 'POS': '121.1743, -21.5733', 'SIZE':'3','Coordinates':"Galactic","Grid":1}
params = {'survey': 'dss', 'POS': '10.684708000000002, 41.26875', 'SIZE':'3'}
r = requests.get('https://skyview.gsfc.nasa.gov/cgi-bin/vo/sia.pl', params=params)


### Looking at the result

The response content is in XML in a VO Table standard:

In [162]:
print(r.url)
# .content returns type str while .text returns type unicode.
print(r.content) 

https://skyview.gsfc.nasa.gov/current/cgi/vo/sia.pl?survey=dss&POS=10.684708000000002%2c+41.26875&SIZE=3
<?xml version="1.0"?>
<VOTABLE version="1.2" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
 xmlns="http://www.ivoa.net/xml/VOTable/v1.2">
  <RESOURCE type="results">
  <PARAM value="10.684708000000002, 41.26875"         datatype="double" arraysize="2" name="INPUT:POS">
    <DESCRIPTION>Center of image to be generated</DESCRIPTION></PARAM>
  <PARAM value="3"        datatype="double" arraysize="*" name="INPUT:SIZE">
    <DESCRIPTION>Size of image in degrees, use 0 to get default</DESCRIPTION></PARAM>
  <PARAM value="Image/FITS,Image/JPEG"      datatype="double" arraysize="*" name="INPUT:FORMAT">
    <DESCRIPTION>Output format requested</DESCRIPTION></PARAM>
  <PARAM value="300,300"      datatype="int" arraysize="*" name="INPUT:NAXIS">
    <DESCRIPTION>Number of pixels in image, use 0 to get default sized pixels (SIZE>0)</DESCRIPTION></PARAM>
  <PARAM value="Tan"  datatype="cha

### Reading the VO table object

In [193]:
from astropy.io import votable 
import tempfile
### Why doesn't this work?  It has to have an actual file to read?  It says "file-like object", but I can't see how.
#vot=votable.parse(r.content)
### Seriously? This is what https://michaelmommert.wordpress.com/2017/02/13/accessing-the-gaia-and-pan-starrs-catalogs-using-python/
### does, but there has to be something less stupid.
#out=open('tmp.xml','w')
#out.write(r.content)
#out.close()
#vot=votable.parse('tmp.xml')
### Only slightly less stupid:
tmp=tempfile.TemporaryFile()
tmp.write(r.content)
vot=votable.parse(tmp)
tmp.close()

Look at what the vot object contains:

In [194]:
###  Just look at the fields (and parameters)
for f in vot.iter_fields_and_params():
        print f


<PARAM ID="INPUT_POS" arraysize="2" datatype="double" name="INPUT:POS" value="[ 10.684708  41.26875 ]"/>
<PARAM ID="INPUT_SIZE" arraysize="*" datatype="double" name="INPUT:SIZE" value="[3.0]"/>
<PARAM ID="INPUT_FORMAT" arraysize="*" datatype="double" name="INPUT:FORMAT" value="[-- --]"/>
<PARAM ID="INPUT_NAXIS" arraysize="*" datatype="int" name="INPUT:NAXIS" value="[300 300]"/>
<PARAM ID="INPUT_PROJ" arraysize="*" datatype="char" name="INPUT:PROJ" value="Tan"/>
<PARAM ID="INPUT_CFRAME" arraysize="*" datatype="char" name="INPUT:CFRAME" value="ICRS"/>
<PARAM ID="INPUT_EQUINOX" datatype="double" name="INPUT:EQUINOX" value="2000.0"/>
<PARAM ID="INPUT_INTERPOLATION" arraysize="*" datatype="char" name="INPUT:INTERPOLATION" value="Clip"/>
<PARAM ID="INPUT_big" datatype="double" name="INPUT:big" value="10000.0"/>
<PARAM ID="INPUT_small" datatype="double" name="INPUT:small" value="5.0"/>
<FIELD ID="Survey" arraysize="*" datatype="char" name="Survey" ucd="VOX:Image_Title"/>
<FIELD ID="Ra" dataty

Extract the fields you're interested in, e.g., the URL of the images made by skyview:

In [195]:
### Since there could be multiple tables in the XML, 
table = vot.get_first_table()
for f in table.fields:
        print f.ID
###  The data is a numpy record array that you can access like this, for example:
#print table.array[1][8]
#  Or like this
print table.array[1]['URL']

### To download the FITS image itself:
print table.array[0]['URL']

Survey
Ra
Dec
Dim
Size
Scale
Format
PixFlags
URL
LogicalName
https://skyview.gsfc.nasa.gov/cgi-bin/images?position=10.684708000000002%2C+41.26875&survey=dss&pixels=300%2C300&sampler=Clip&size=3.0%2C3.0&projection=Tan&coordinates=J2000.0&requestID=skv1510856809053&nofits=1&quicklook=jpeg&return=jpeg
https://skyview.gsfc.nasa.gov/cgi-bin/images?position=10.684708000000002%2C+41.26875&survey=dss&pixels=300%2C300&sampler=Clip&size=3.0%2C3.0&projection=Tan&coordinates=J2000.0&requestID=skv1510856809053&return=FITS


### Resulting image
You can display it in python by
(give it a few seconds to appear)

In [196]:
from IPython.display import Image, display
from IPython.core.display import HTML 
display(Image(url=table.array['URL'][1]))

# Example 3:  HEASARC catalog as VOTable
(This is fetching the entire chanmaster catalog of 17k lines.  Give it a few seconds.)

In [197]:
payload3 = {'name': 'chanmaster'}
chan = requests.get('http://heasarc.gsfc.nasa.gov/cgi-bin/W3Browse/getvotable.pl', params=payload3)
tmp=tempfile.TemporaryFile()
tmp.write(chan.content)
chan_vot=votable.parse(tmp)
tmp.close()
chan_table = chan_vot.get_first_table()


In [198]:
chan_table

<Table masked=True length=17701>
 col1     col2             col3         ...            col19            col20
                                        ...                                  
int32    str20            str50         ...            str50            int16
----- ----------- --------------------- ... --------------------------- -----
18911 untriggered          T Tauri Star ...                STARS AND WD  9999
18922 untriggered         FUor Outburst ...                STARS AND WD  9999
18923 untriggered         FUor Outburst ...                STARS AND WD  9999
18924 untriggered         FUor Outburst ...                STARS AND WD  9999
18956 untriggered            DECam-EM-1 ...          BH AND NS BINARIES  1000
18957 untriggered            DECam-EM-1 ...          BH AND NS BINARIES  1000
18963 untriggered New INTEGRAL source 1 ...          BH AND NS BINARIES  1000
18964 untriggered New INTEGRAL source 2 ...          BH AND NS BINARIES  1000
18965 untriggered           New

# Example 4:  HEASARC spectra retrieval

In [202]:
###  Example from 
###  https://heasarc.gsfc.nasa.gov/docs/archive/vo/instructions.html#_accessing_ssa
###  which apparently causes an error in votable.parse().  Interesting.  I get:
### 
###  ValueError: None:77:10: ValueError: invalid literal for int() with base 10: '12.0' (in row 0, col 'SSA_datalength')
###  
#payload3 = {'table': 'chanmaster',"POS":"187.27,2.05","SIZE":".01", "REQUEST":"queryData"}
payload3 = {'table': 'chanmaster',"POS":"187.27,2.05","SIZE":".01", "REQUEST":"queryData"}
spec = requests.get('https://heasarc.nasa.gov/xamin/vo/ssa', params=payload3)
tmp=tempfile.TemporaryFile()
tmp.write(spec.content)
spec_vot=votable.parse(tmp)
tmp.close()
spec_table = spec_vot.get_first_table()

In [203]:
for f in spec_table.fields:
        print f.ID

obsid
status
name
ra
dec
time
detector
grating
exposure
type
pi
public_date
SSA_datalength
SSA_datamodel
SSA_title
SSA_instrument
SSA_publisher
SSA_tmid
SSA_stop_time
SSA_start_time
SSA_duration
SSA_bandpass
SSA_wavelength_max
SSA_wavelength_min
SSA_bandwidth
SSA_ra
SSA_dec
SSA_coord_obs
SSA_fov
SSA_reference
SSA_format


In [204]:
print spec_table

obsid status name  ra dec ... SSA_coord_obs SSA_fov SSA_reference SSA_format
                  deg deg ...      deg        deg                           
----- ------ ---- --- --- ... ------------- ------- ------------- ----------


DON'T USE THIS!  Trying to use simple XML tools.  Hadn't discovered astropy.io.votable.

In [39]:
from xml.etree import ElementTree
tree = ElementTree.fromstring(r.content)
for child in tree:
        print "1 tag '{}'".format(child.tag)
        print "1 attrib '{}'".format(child.attrib)
        for child2 in child:
            print "2 tag '{}'".format(child2.tag)
            print "2 attr '{}'".format(child2.attrib)

1 tag '{http://www.ivoa.net/xml/VOTable/v1.2}RESOURCE'
1 attrib '{'type': 'results'}'
2 tag '{http://www.ivoa.net/xml/VOTable/v1.2}PARAM'
2 attr '{'datatype': 'double', 'arraysize': '2', 'value': '270.67,-22.97', 'name': 'INPUT:POS'}'
2 tag '{http://www.ivoa.net/xml/VOTable/v1.2}PARAM'
2 attr '{'datatype': 'double', 'arraysize': '*', 'value': '3', 'name': 'INPUT:SIZE'}'
2 tag '{http://www.ivoa.net/xml/VOTable/v1.2}PARAM'
2 attr '{'datatype': 'double', 'arraysize': '*', 'value': 'Image/FITS,Image/JPEG', 'name': 'INPUT:FORMAT'}'
2 tag '{http://www.ivoa.net/xml/VOTable/v1.2}PARAM'
2 attr '{'datatype': 'int', 'arraysize': '*', 'value': '300,300', 'name': 'INPUT:NAXIS'}'
2 tag '{http://www.ivoa.net/xml/VOTable/v1.2}PARAM'
2 attr '{'datatype': 'char', 'arraysize': '*', 'value': 'Tan', 'name': 'INPUT:PROJ'}'
2 tag '{http://www.ivoa.net/xml/VOTable/v1.2}PARAM'
2 attr '{'datatype': 'char', 'arraysize': '*', 'value': 'ICRS', 'name': 'INPUT:CFRAME'}'
2 tag '{http://www.ivoa.net/xml/VOTable/v1.2}P

In [49]:
for child in tree.iter('TABLE'):
    print child.find('FIELD').attrib